In [1]:
import numpy as np
from sklearn.datasets import make_blobs

In [3]:
X, y = make_blobs(n_samples=1000, centers=2, random_state=200)

In [5]:
import torch
import torch.nn as nn

In [6]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
y = y*2 - 1

In [14]:
class DualSVC():
    def __init__(self, n_samples, gamma=50, iters=50000, lr=1e-5):
        self.n_samples = n_samples
        self.gamma = gamma
        self.alpha = nn.Parameter(torch.rand(n_samples))
        self.iters = iters
        self.lr = lr
    
    def _loss(self, x, y):
        loss1 = -self.alpha.sum()
        loss2 = 0.5 * (torch.matmul(x.T, (self.alpha * y))**2).sum()
        loss3 = self.gamma * (torch.dot(self.alpha, y)**2)
        loss = loss1 + loss2 + loss3
        return loss
    
    def _zero_grad(self):
        if self.alpha.grad is not None:
            self.alpha.grad.zero_()
    
    def _update_weights(self):
        self.alpha.data -= self.lr * self.alpha.grad.data
    
    def _clip_weights(self):
        self.alpha.data = torch.clamp(self.alpha.data, min=0)
    
    def fit(self, X, y):
        for i in range(self.iters):
            self._zero_grad()
            self._clip_weights()
            loss = self._loss(X, y)
            loss.backward()
            self._update_weights()
        
        with torch.no_grad():
            self._clip_weights()
            idx = torch.where(self.alpha.data>0)[0]
            self.w = torch.matmul(X[idx].T, self.alpha.data[idx] * y[idx])
            self.b = (y[idx] - torch.matmul(X[idx], self.w)).mean()
    
    def predict(self, data):    
        with torch.no_grad():
            score = torch.matmul(data, self.w) + self.b
            pred = (score > 0).to(data.dtype)
            pred = pred * 2 - 1
        
        return pred, score

In [15]:
N = len(X)
clf = DualSVC(N, iters=200000)

In [16]:
clf.fit(X, y)

In [17]:
pred, score = clf.predict(X)